In [397]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy import stats
from scipy.stats import linregress
from api_keys import api_key

In [64]:
music_df = pd.read_csv("resources/spotify-2023.csv", encoding='latin-1')
music_df

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,My Mind & Me,Selena Gomez,1,2022,11,3,953,0,91473363,61,...,144,A,Major,60,24,39,57,0,8,3
949,Bigger Than The Whole Sky,Taylor Swift,1,2022,10,21,1180,0,121871870,4,...,166,F#,Major,42,7,24,83,1,12,6
950,A Veces (feat. Feid),"Feid, Paulo Londra",2,2022,11,3,573,0,73513683,2,...,92,C#,Major,80,81,67,4,0,8,6
951,En La De Ella,"Feid, Sech, Jhayco",3,2022,10,20,1320,0,133895612,29,...,97,C#,Major,82,67,77,8,0,12,5


In [65]:
music_df = music_df.dropna(how='any')
music_df

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,My Mind & Me,Selena Gomez,1,2022,11,3,953,0,91473363,61,...,144,A,Major,60,24,39,57,0,8,3
949,Bigger Than The Whole Sky,Taylor Swift,1,2022,10,21,1180,0,121871870,4,...,166,F#,Major,42,7,24,83,1,12,6
950,A Veces (feat. Feid),"Feid, Paulo Londra",2,2022,11,3,573,0,73513683,2,...,92,C#,Major,80,81,67,4,0,8,6
951,En La De Ella,"Feid, Sech, Jhayco",3,2022,10,20,1320,0,133895612,29,...,97,C#,Major,82,67,77,8,0,12,5


In [9]:
#rapid api deezer
# Set up your rapidAPI key and URL for deezer
headers = {
    "X-RapidAPI-Key": "api_key",
	"X-RapidAPI-Host": "deezerdevs-deezer.p.rapidapi.com"
}

url = "https://deezerdevs-deezer.p.rapidapi.com/search"

# Create an empty list to store the deezer album IDs:
d_album_id= []

# Loop through the DataFrame
for index, row in music_df.iterrows():
    title = row['track_name']

    # Set up the parameters for the API call
    querystring = {"q":title}
    
    #print("Processing track:",title)
    # Make the API request
    response = requests.get(url, headers=headers, params=querystring)
    
    # Check if the request was successful
    if response.status_code == 200:
        data_response = response.json()
        pretty_data = json.dumps(data_response, indent=2)
        #print(data_response.keys())
        #print(pretty_data)
        #check if data is a list with at least one item
        if "data" in data_response and len(data_response["data"]) > 0:
            # Extract the album id from the API response
            album_id = int(data_response["data"][0]["album"]["id"])
            #print(album_id)
            d_album_id.append(album_id)
        else:
            #print(track_name, "not found")
            d_album_id.append(0)
    else:
            #print("Error processing track", album_id)
            d_album_id.append(0)


In [11]:
# Add the 'd_album_id' column to the DataFrame
music_df['Deezer Album ID'] = d_album_id

# Print the DataFrame to see the results
music_df.tail(20)

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,Deezer Album ID
933,Vigilante Shit,Taylor Swift,1,2022,10,21,1948,0,253650850,12,...,E,Minor,80,16,28,17,0,12,39,368474187
934,Question...?,Taylor Swift,1,2022,10,21,1608,0,223064273,10,...,G,Major,75,11,50,20,0,30,17,368474187
935,On BS,"Drake, 21 Savage",2,2022,11,4,1338,0,170413877,9,...,A,Major,84,33,36,2,0,39,59,372625207
936,Mastermind,Taylor Swift,1,2022,10,21,1936,0,218320587,7,...,E,Major,66,12,35,55,0,9,14,368474187
937,Circo Loco,"Drake, 21 Savage",2,2022,11,4,1794,0,141720999,26,...,C#,Major,73,25,61,1,0,32,7,372625207
938,Labyrinth,Taylor Swift,1,2022,10,21,1597,0,187339835,6,...,NaN,Major,48,15,31,80,22,12,4,368474187
939,Spin Bout U,"Drake, 21 Savage",2,2022,11,4,1652,2,198365537,26,...,G,Major,77,20,70,1,0,16,5,372625207
940,Sweet Nothing,Taylor Swift,1,2022,10,21,1747,0,186104310,9,...,NaN,Major,34,39,16,97,0,12,5,5969222
941,"Would've, Could've, Should've",Taylor Swift,1,2022,10,21,1715,0,177503916,4,...,G,Major,48,55,84,43,0,15,12,368506677
942,Con La Brisa,"Ludwig Goransson, Foudeqush",2,2022,11,4,486,0,71095708,8,...,D,Minor,62,25,44,51,33,14,3,373539777


In [12]:
#export to csv:
music_df.to_csv("resources/album_id.csv", encoding = "Latin-1", index=False, header=True)

In [13]:
#read in saved data
music_df=pd.read_csv("resources/album_id.csv", encoding = "Latin-1")
music_df.head()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,Deezer Album ID
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,B,Major,80,89,83,31,0,8,4,463574485
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,C#,Major,71,61,74,7,0,10,4,410083687
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,F,Major,51,32,53,17,0,31,6,484372295
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,A,Major,55,58,72,11,0,11,15,108447472
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,A,Minor,65,23,80,14,63,11,6,442984025


In [35]:
#rapid api deezer to find genre based on album id
# Set up your rapidAPI key and URL for deezer
headers = {
	"X-RapidAPI-Key": "api_key",
	"X-RapidAPI-Host": "deezerdevs-deezer.p.rapidapi.com"
}

# Create an empty list to store the deezer album IDs:
d_genre= []

# Loop through the DataFrame
for index, row in music_df.iterrows():
    deezer_id = row['Deezer Album ID']

    url = "https://deezerdevs-deezer.p.rapidapi.com/album/"+ str(deezer_id)

    #print("Processing album:", deezer_id)
    # Make the API request
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        id_response = response.json()
        pretty_data = json.dumps(id_response, indent=2)
 
        #check if data is a list with at least one item
        try:
            # Extract the album id from the API response
            genre_find = id_response["genres"]["data"][0]["name"]
            d_genre.append(genre_find)
            #print(genre_find)
        except:
            #print(track_name, "not found")
            d_genre.append(None)
    else:
            #print("Error processing album", deezer_id)
            d_genre.append(None)


In [42]:
# Add the 'd_album_id' column to the DataFrame
music_df['Deezer Genre'] = d_genre

# Print the DataFrame to see the results
music_df.head(10)

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,Deezer Album ID,Deezer Genre
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,Major,80,89,83,31,0,8,4,463574485,Asian Music
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,Major,71,61,74,7,0,10,4,410083687,Rap/Hip Hop
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,Major,51,32,53,17,0,31,6,484372295,Pop
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,Major,55,58,72,11,0,11,15,108447472,Pop
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,Minor,65,23,80,14,63,11,6,442984025,Latin Music
5,Sprinter,"Dave, Central Cee",2,2023,6,1,2186,91,183706234,67,...,Major,92,66,58,19,0,8,24,447914165,Rap/Hip Hop
6,Ella Baila Sola,"Eslabon Armado, Peso Pluma",2,2023,3,16,3090,50,725980112,34,...,Minor,67,83,76,48,0,8,3,418111007,Traditional Mexicano
7,Columbia,Quevedo,1,2023,7,7,714,43,58149378,25,...,Major,67,26,71,37,0,11,4,459659375,Reggaeton
8,fukumean,Gunna,1,2023,5,15,1096,83,95217315,60,...,Minor,85,22,62,12,0,28,9,453946145,Rap/Hip Hop
9,La Bebe - Remix,"Peso Pluma, Yng Lvcas",2,2023,3,17,2953,44,553634067,49,...,Minor,81,56,48,21,0,8,33,382919757,Rap/Hip Hop


In [43]:
#export to csv:
music_df.to_csv("resources/all_data_genre.csv", encoding = "Latin-1", index=False, header=True)

In [335]:
#read the saved data:
music_df=pd.read_csv("resources/all_data_genre.csv", encoding = "Latin-1")
music_df.head()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,Deezer Album ID,Deezer Genre
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,Major,80,89,83,31,0,8,4,463574485,Asian Music
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,Major,71,61,74,7,0,10,4,410083687,Rap/Hip Hop
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,Major,51,32,53,17,0,31,6,484372295,Pop
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,Major,55,58,72,11,0,11,15,108447472,Pop
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,Minor,65,23,80,14,63,11,6,442984025,Latin Music


# Question 1- 